Importing libraries

In [1]:
import google.generativeai as genai
from langchain.embeddings.base import Embeddings
import numpy as np
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
#from langchain_classic.chains.combine_documents import create_stuff_documents_chain
#from langchain_classic.chains import create_retrieval_chain
import os
from dotenv import load_dotenv
from langchain_classic.memory import ConversationBufferMemory


load_dotenv()

c:\python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

Getting API key

In [2]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.5-flash')

Preprocessing for RAG


In [3]:
loader=PyPDFLoader("C:/python/langchain/RAG/CUSTOMER-SERVICE-2014.pdf")
docs=loader.load()

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

In [ ]:
texts = [doc.page_content for doc in documents]
texts

Embeddings

In [ ]:
#embeddings=configure.models.embed_content(model="gemini-embedding-001")

embeddings = genai.embed_content(
    model="models/text-embedding-004",
    content=texts)
embeddings["embedding"]

In [7]:
vec_np = np.array(embeddings["embedding"]).astype('float32')
vec_np.shape

(36, 768)

In [8]:
dimension = len(vec_np[0])
index = faiss.IndexFlatL2(dimension)
index.add(vec_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001898A465B90> >

Sample query

In [ ]:
query = "what is the purpose of FD"
query_embedding = genai.embed_content(
    model="models/text-embedding-004",
    content=query)
query_vector = np.array(query_embedding["embedding"]).astype('float32')
query_vector

In [10]:
D, I = index.search(np.array([query_vector]), 3) #I = index num of best 3 matching docs and D similarity score (best < worst)
print(D, I)

[[1.1303391 1.1714585 1.2059507]] [[2 8 0]]


In [ ]:
retireved_docs =[documents[i] for i in I[0]]
final_docs=[]
for doc in retireved_docs:
    print(doc.page_content)
    print("----------------------------------------------------------------------------------------------------------------")
    final_docs.append(doc.page_content)

Conversation memory

In [12]:
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

C:\Users\2425586\AppData\Local\Temp\ipykernel_31392\3749131566.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
# Load variables for use in prompts
variables = memory.load_memory_variables({})
print(variables["chat_history"])

[]


In [14]:
def embed(input_text):
    #embedding the input_text
    query_embedding = genai.embed_content(
    model="models/text-embedding-004",
    content=input_text)
    query_vector = np.array(query_embedding["embedding"]).astype('float32')
    D, I = index.search(np.array([query_vector]), 3)
    retireved_docs =[documents[i] for i in I[0]]
    final_docs=[]
    for doc in retireved_docs:
        final_docs.append(doc.page_content)
    context = "\n".join(final_docs)
    return context


In [ ]:
while True:
    input_text = input("Enter your text here:")
    context = embed(input_text)

    if input_text.lower() == "exit" or input_text.lower() == "quit":
        break
    else:
        #prompt = f"you are an helpful AI assisstant, so respond accordingly {input_text}"
        prompt = f"""
        you are a helpful AI assisstant,so speak accordingly. 
        use the below context to answer the question.
        PREVIOUS MEMORY:{memory}
        CONTEXT: {context}
        QUESTION: {input_text}
        ANSWER:"""
        response = model.generate_content(prompt)
        print("User🙍🏻: ",input_text)
        #print("Context📚",context)
        print("AI🤖: ",response.text)
        memory.chat_memory.add_user_message(input_text)
        memory.chat_memory.add_ai_message(response.text)
